# Advent of Code Day 18

For Day 18, we build a basic instruction processor.  For part one, there are a handful of instructions that just have to be implemented to manipulate some registers (held in a dictionary) with the goal of looking for a specific instruction to execute. Part two revises the nature of two instructions and requires building (in essence) a cheap multiprocessing.  The second part was tricky in terms of getting the coordination and termination conditions exactly right.  

In [ ]:
from utils import read_input
from collections import defaultdict
import Queue

### Instruction Methods

Given an instruction as a string, parse_instruction returns a tuple consisting of the op (taken directly from the instruction) and the arguments (as a list).  The arguments can be values or registers, but it's up to a later component to interpret them.

Get_argument_value takes the application registers (a dictionary) and a particular instruction argument and returns *either* the arg itself (when the arg represents a value) or the value of the register (when the arg represents a register). This determination is based on whether the arg is numeric (a value) or alphabetic (a register).

In [ ]:
def parse_instruction(instruction):
    op = instruction[0:3]
    
    args = [ins.strip() for ins in instruction[4:].split(' ')]
    
    return (op, args)


def get_argument_value(registers, arg):
    try:
        numeric_arg = int(arg)        
        return numeric_arg
    except:
        return registers[arg] 

### Part One

Part one is straight-forward enough - only the snd/rcv instructions require any special handling.  The heart of the logic is using get_argument_value to retrieve the value that should be applied.  This could be the value from a register or a value read in from the instructions directly - get_argument_value abstracts both of that away.  The caller only has to be certain that the argument should be treated in such a way and not as a register directly.  

In [ ]:
def execute_instructions(instructions):
    
    registers = defaultdict(int)
    instruction_index = 0
    last_frequency = None
        
    while instruction_index < len(instructions):    
        
        instruction = instructions[instruction_index]
        (op, args) = parse_instruction(instruction)
      
        if op == 'set':
            registers[args[0]] = get_argument_value(registers, args[1])
            instruction_index += 1
        elif op == 'add':
            registers[args[0]] += get_argument_value(registers, args[1])
            instruction_index += 1
        elif op == 'mul':
            registers[args[0]] *= get_argument_value(registers, args[1])
            instruction_index += 1
        elif op == 'mod':
            registers[args[0]] %= get_argument_value(registers, args[1])
            instruction_index += 1
        elif op == 'snd':
            last_frequency = get_argument_value(registers, args[0])
            instruction_index += 1
        elif op == 'jgz':
            if get_argument_value(registers, args[0]) > 0:
                instruction_index += get_argument_value(registers, args[1])
            else:
                instruction_index += 1
        elif op == 'rcv':
            print 'Recovered Frequency = {}'.format(last_frequency)
            return        

### Part 2

Part 2 was a challenge because it requires interleaving two distinct "programs" and finding a way for them to communicate.  I decided to use a Queue object for each program.  Each program also gets its own registers and instruction pointer.  Lastly, each needs to be able to communicate if it's completed its instructions or if its blocked.  The problem description mentions that the program should work correctly regardless of how you choose to run the programs.  I decided to have each program run up until the point it encountered a rcv instruction and its queue was empty.  At that point, its blocked and transfers execution to the other program.  Deadlocks can occur if both programs are blocked, or if one program is done and the other is blocked, in which event the combined program needs to terminate. 

In [ ]:
def execute_instructions2(instructions):
    
    # Each program needs its own registers and the 'p' register starts as the Program ID (0 or 1)
    registers = [defaultdict(int), defaultdict(int)]
    registers[0]['p'] = 0
    registers[1]['p'] = 1    
    me = 0
    other = 1
    
    count = 0
    # each program needs its own instruction pointer
    instruction_index = [0, 0]
    
    # We need to track both each program being done (completed all instructions) and blocked (waiting to receive a valueP)
    done = [False, False]
    blocked = [False, False]
    
    # each program needs a queue.  We'll say that each program reads from the queue at
    # queues[ProgramID] i.e. that a program places an item onto the other program's queue
    queues = [Queue.Queue(), Queue.Queue()]
           
    while True:       
        
        done[0] = instruction_index[0] == len(instructions)
        done[1] = instruction_index[1] == len(instructions)
        
        if all(done):
            print 'Both programs are done - exiting normally'
            break
            
        if all(blocked) and all([q.empty for q in queues]):
            print 'Both programs are blocked with no input queued - terminating due to deadlock'
            break
        
        instruction = instructions[instruction_index[me]]
        (op, args) = parse_instruction(instruction)
        
        if op == 'set':
            registers[me][args[0]] = get_argument_value(registers[me], args[1])
            instruction_index[me] += 1
        elif op == 'add':
            registers[me][args[0]] += get_argument_value(registers[me], args[1])
            instruction_index[me] += 1
        elif op == 'mul':
            registers[me][args[0]] *= get_argument_value(registers[me], args[1])
            instruction_index[me] += 1
        elif op == 'mod':
            registers[me][args[0]] %= get_argument_value(registers[me], args[1])
            instruction_index[me] += 1      
        elif op == 'jgz':
            if get_argument_value(registers[me], args[0]) > 0:
                instruction_index[me] += get_argument_value(registers[me], args[1])
            else:
                instruction_index[me] += 1
        elif op == 'snd': # send
            value = get_argument_value(registers[me], args[0])
            queues[other].put(value)
            instruction_index[me] += 1           
            blocked[other] = False # if I send you a value, you shouldn't be considered blocked
            
            if me == 1:
                count += 1
        elif op == 'rcv':
            if queues[me].empty():             
                print '{} will be blocked'.format(me)
                blocked[me] = True
                me, other = other, me # Just because I'm blocked doesn't mean the other guy is.
            else:
                blocked[me] = False 
                value = queues[me].get()
                register = args[0]
                registers[me][register] = value
                instruction_index[me] += 1
            
                
    if all(blocked):
        print 'Everyone is blocked - deadlock'
        
    if done[0] and blocked[1]:
        print '0 is done but 1 is blocked - deadlock'
        
    if done[1] and blocked[0]:
        print '1 is done but 0 is blocked - deadlock'
        
    if all(done):
        print 'Everyone finished normally'
        
    print 'Program 1 sent {} values'.format(count)
      
    

In [ ]:
instructions = read_input('Input/day18.txt')

execute_instructions2(instructions)

